<a href="https://colab.research.google.com/github/peteralexandercharles/AIO-Boot/blob/master/End-to-end-Text-to-Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/jaywalnut310/vits.git

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 81 (delta 22), reused 66 (delta 12), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [ ]:
%cd vits

/content/vits


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 183 kB 46.6 MB/s 
     |████████████████████████████████| 11.6 MB 12.7 MB/s 
     |████████████████████████████████| 20.1 MB 1.7 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 25.9 MB 7.5 kB/s 
     |████████████████████████████████| 6.8 MB 15.5 MB/s 
     |████████████████████████████████| 748.8 MB 17 kB/s 
     |████████████████████████████████| 5.9 MB 18.9 MB/s 
     |████████████████████████████████| 238 kB 56.9 MB/s 
     |████████████████████████████████| 195 kB 55.6 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201395 sha256=d25fb42cda2815a32e455bc4089ec74c82b45d0b1f0cf296ece33d1715572ea8
  Stored in directory: /root/.cache/pip/wheels/de/1e/aa/d91797ae7e1ce11853ee100bee9d1781ae9d750e7458c95afb
Successfully built librosa
  Attempting uninstal

In [ ]:
%cd vits  # If you restart runtime
%cd monotonic_align

[Errno 2] No such file or directory: 'vits # If you restart runtime'
/content/vits
/content/vits/monotonic_align


In [ ]:
!python setup.py build_ext --inplace

Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c core.c -o build/temp.linux-x86_64-3.7/core.o
creating /content/vits/mono

In [ ]:
!sudo apt-get install espeak -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 40 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 

In [ ]:
%cd ..

/content/vits


In [ ]:
!pip install gdown

In [ ]:
!gdown 'https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT'

Downloading...
From: https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT
To: /content/vits/pretrained_ljs.pth
146MB [00:01, 73.7MB/s]


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_ljs.pth", net_g, None)

In [ ]:
stn_tst = get_text("We propose VITS, Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech.", hps)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

## Multi Speaker

In [ ]:
!gdown 'https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru'

In [ ]:
hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")

In [ ]:
net_g_ms = SynthesizerTrn(
    len(symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_vctk.pth", net_g_ms, None)

In [ ]:
sid = torch.LongTensor([4]) # speaker identity
stn_tst = get_text("We propose VITS, Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech.", hps_ms)

with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate))

## Voice Conversion

In [ ]:
!wget https://jaywalnut310.github.io/vits-demo/wavs/vc_01_01.wav

In [ ]:
from mel_processing import spectrogram_torch
from utils import load_wav_to_torch

audio, sampling_rate = load_wav_to_torch("./vc_01_01.wav")
        
y = audio / hps_ms.data.max_wav_value
y = y.unsqueeze(0)

spec = spectrogram_torch(y, hps_ms.data.filter_length,
    hps_ms.data.sampling_rate, hps_ms.data.hop_length, hps_ms.data.win_length,
    center=False)
spec_lengths = torch.LongTensor([spec.size(-1)])
sid_src = torch.LongTensor([81])

In [ ]:
with torch.no_grad():
    sid_tgt1 = torch.LongTensor([77])
    sid_tgt2 = torch.LongTensor([14])
    sid_tgt3 = torch.LongTensor([1])
    sid_tgt4 = torch.LongTensor([17])
    audio1 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.float().numpy()
    audio2 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.float().numpy()
    audio3 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.float().numpy()
    audio4 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt4)[0][0,0].data.float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].numpy(), rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt4.item())
ipd.display(ipd.Audio(audio4, rate=hps_ms.data.sampling_rate))

Original SID: 81


Converted SID: 77


Converted SID: 14


Converted SID: 1


Converted SID: 17
